In [1]:
import pandas as pd
import numpy as np

#Reading data
df2014=pd.read_csv("../2014_Financial_Data.csv")
df2015=pd.read_csv("../2015_Financial_Data.csv")
df2016=pd.read_csv("../2016_Financial_Data.csv")
df2017=pd.read_csv("../2017_Financial_Data.csv")
df2018=pd.read_csv("../2018_Financial_Data.csv")

#Changing column name for price variation into same name
df2014.rename(columns={"2015 PRICE VAR [%]": "PRICE VAR NEXT YEAR"},inplace=True)
df2015.rename(columns={"2016 PRICE VAR [%]": "PRICE VAR NEXT YEAR"},inplace=True)
df2016.rename(columns={"2017 PRICE VAR [%]": "PRICE VAR NEXT YEAR"},inplace=True)
df2017.rename(columns={"2018 PRICE VAR [%]": "PRICE VAR NEXT YEAR"},inplace=True)
df2018.rename(columns={"2019 PRICE VAR [%]": "PRICE VAR NEXT YEAR"},inplace=True)

#Adding year attribute for each csv file
df2014['Year']=2014
df2015['Year']=2015
df2016['Year']=2016
df2017['Year']=2017
df2018['Year']=2018

data = pd.concat([df2014 ,df2015, df2016, df2017, df2018])
data.rename(columns={'Unnamed: 0':'Stock'}, inplace=True)

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22077 entries, 0 to 4391
Columns: 226 entries, Stock to Year
dtypes: float64(222), int64(2), object(2)
memory usage: 38.2+ MB


,Stock,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,...,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector,PRICE VAR NEXT YEAR,Class,Year
0,PG,7.440100e+10,-0.0713,3.903000e+10,3.537100e+10,0.000000e+00,2.146100e+10,2.146100e+10,1.391000e+10,7.090000e+08,...,-0.0217,0.0359,0.0316,0.1228,0.0000,-0.1746,Consumer Defensive,-9.323276,0,2014
1,VIPS,3.734148e+09,1.1737,2.805625e+09,9.285226e+08,1.083303e+08,3.441414e+08,7.939267e+08,1.345959e+08,1.214869e+07,...,NaN,NaN,NaN,NaN,1.6484,1.7313,Consumer Defensive,-25.512193,0,2014
2,KR,9.837500e+10,0.0182,7.813800e+10,2.023700e+10,0.000000e+00,1.519600e+10,1.751200e+10,2.725000e+09,4.430000e+08,...,0.0981,0.1886,0.3268,0.2738,0.0000,0.0234,Consumer Defensive,33.118297,1,2014
3,RAD,2.552641e+10,0.0053,1.820268e+10,7.323734e+09,0.000000e+00,6.561162e+09,6.586482e+09,7.372520e+08,4.245910e+08,...,-0.0510,-0.0189,0.1963,-0.0458,0.0000,-0.0060,Consumer Defensive,2.752291,1,2014
4,GIS,1.790960e+10,0.0076,1.153980e+10,6.369800e+09,0.000000e+00,3.474300e+09,3.412400e+09,2.957400e+09,3.024000e+08,...,0.0090,0.0215,0.0274,0.1025,0.0000,-0.0220,Consumer Defensive,12.897715,1,2014


In [2]:
data.dtypes #looking column types

Stock                    object
Revenue                 float64
Revenue Growth          float64
Cost of Revenue         float64
Gross Profit            float64
                         ...   
SG&A Expenses Growth    float64
Sector                   object
PRICE VAR NEXT YEAR     float64
Class                     int64
Year                      int64
Length: 226, dtype: object

In [3]:
attributeInfo = pd.DataFrame(columns = ["Feature name","Description", 
                                   "Type", "Min", "Max", "Avg.", "Std.Dev.", "Entropy", "# of values", "missing values %"]
                            )
print(attributeInfo) #dataframe for storing column informations

Empty DataFrame
Columns: [Feature name, Description, Type, Min, Max, Avg., Std.Dev., Entropy, # of values, missing values %]
Index: []


In [4]:
from scipy.stats import entropy

def findDataType(dataType,colName): # {nominal, numeric_integer, numeric_continuous, boolean, text}
    if dataType=="float64":
        return "numeric_continuous"
    elif colName=="Year":
        return "nominal"
    elif colName=="Stock":
        return "text" 
    elif colName=="Sector":
        return "nominal"
    elif colName=="Class":
        return "boolean"

i=0

missingPercentage = data.isnull().sum() * 100 / len(data) # missing val % for each column

for col in data.columns: #calculate and put requested information into dataframe for each column
    featureName=col
    description="" #descriptions handled by hand
    dataType=findDataType(data.dtypes[col],col)
    if dataType=="text":
        minVal=maxVal=avgVal=stdDev=entropyVal=""
    elif dataType=="nominal":
        avgVal=stdDev=""
        maxVal=data[col].value_counts().index[0]
        minVal=data[col].value_counts().index[-1]
        value,counts = np.unique(data[col], return_counts=True)
        entropyVal=entropy(counts, base=None)

    elif dataType=="boolean":
        minVal=maxVal=avgVal=stdDev=entropyVal=""
        
    else:
        minVal=data[col].min()
        maxVal=data[col].max()
        avgVal=data[col].mean()
        stdDev=data[col].std()
        entropyVal=0
        value,counts = np.unique(data[col], return_counts=True)
        entropyVal=entropy(counts, base=None)
    
    total=data[col].count()
    missVal=round(missingPercentage[col],2)
    
    
    attributeInfo.loc[i]=[featureName,description,dataType,minVal,maxVal,avgVal,stdDev,entropyVal,total,missVal]
    i+=1
    
print(attributeInfo.head())

      Feature name Description                Type          Min          Max  \
0            Stock                            text                             
1          Revenue              numeric_continuous -6.27616e+08  1.88689e+12   
2   Revenue Growth              numeric_continuous     -12.7693      42138.7   
3  Cost of Revenue              numeric_continuous -2.98689e+09  1.58153e+12   
4     Gross Profit              numeric_continuous  -1.2808e+10   4.6216e+11   

          Avg.     Std.Dev.  Entropy # of values  missing values %  
0                                          22077              0.00  
1  5.16162e+09  3.19731e+10  9.67218       20906              5.30  
2      3.62221      312.648  8.71248       19989              9.46  
3  3.25857e+09  2.58309e+10  8.32462       20306              8.02  
4  1.97045e+09  8.73575e+09  9.66255       20870              5.47  


In [5]:
attributeInfo.to_excel("attributes.xlsx") #import column information to excel file for report